# 불균형 클래스
- 클래스 불균형에 대한 이해
- 클래스 가중치 사용
- Resampling
- 적절한 평가지표

#### 로지스틱 회귀로

In [121]:
import numpy as np

In [122]:
# 불균형 데이터 (9:1) 양성(0):악성(1)  
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X, y = data.data, data.target

# 악성을 소수클래스로 만들기
print(f'악성 양성의 오리지널 비율: {np.unique(y,return_counts=True)}')
m_index = np.where(y == 1)[0] #악성 인덱스 변환
b_index = np.where( y == 0)[0]

# 악성은 일부만, 양성은 더 많이 사용
# 악성의 30%만 사용, 양성은 전체
size_30 = int(len(m_index) * 0.11)
selected_m_index = np.random.choice(m_index,size=size_30,replace=False)
selected_b_index = b_index

concatenate_selected_index = np.concatenate( [selected_m_index, selected_b_index] )
np.random.shuffle(concatenate_selected_index)

X_lmb = X[concatenate_selected_index]
y_lmb = y[concatenate_selected_index]

# 클래스 분포 확인
unique, counts = np.unique(y_lmb, return_counts=True)
print('클래스 분포')

for label, count in zip(unique, counts):
    percentage =  count/len(y_lmb)
    print(f'클래스 {label} : {count}개 ({percentage:.1f}%)')

악성 양성의 오리지널 비율: (array([0, 1]), array([212, 357]))
클래스 분포
클래스 0 : 212개 (0.8%)
클래스 1 : 39개 (0.2%)


In [123]:
# 불균형 상태로 진행
# 스케일링 정규화 StandardScaler
# pipe
# 평가는 class report

In [124]:
print('< 클래스 불균형 >')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
# np.random.seed(42)

X_train,X_test,y_train,y_test = train_test_split(X_lmb,y_lmb, test_size=0.3, random_state=42)

pipe = Pipeline( [ 
    ('sclaer',StandardScaler()),
    ('clf',LogisticRegression())
])

pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)

print(classification_report(y_test, y_pred ))

< 클래스 불균형 >
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        61
           1       1.00      0.67      0.80        15

    accuracy                           0.93        76
   macro avg       0.96      0.83      0.88        76
weighted avg       0.94      0.93      0.93        76



In [125]:
print('< 클래스 불균형 해결: 클래스 가중치 사용 >')

# 기존 파이프라인 복사 후, 클래스 가중치 적용
from copy import deepcopy

pipe_weight = deepcopy(pipe)
pipe_weight.set_params(clf__class_weight='balanced')

pipe_weight.fit(X_train, y_train)
y_pred = pipe_weight.predict(X_test)

print(classification_report(y_test, y_pred))

# -------------------------
print('< 가중치 계산 >')
n_samples = len(y_train)
n_classes = 2
class_counts = np.bincount(y_train)
for i in range(n_classes):
    weight = n_samples / (n_classes*class_counts[i])
    print(f' 클래스: {i}:{weight:.3f}')

< 클래스 불균형 해결: 클래스 가중치 사용 >
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        61
           1       1.00      0.87      0.93        15

    accuracy                           0.97        76
   macro avg       0.98      0.93      0.96        76
weighted avg       0.97      0.97      0.97        76

< 가중치 계산 >
 클래스: 0:0.579
 클래스: 1:3.646


#### 랜덤포레스트로

In [126]:
print('<클래스 불균형>')

from sklearn.ensemble import RandomForestClassifier

pipe_rf = Pipeline([
    ('sclaer',StandardScaler()),
    ('clf',RandomForestClassifier())
])

pipe_rf.fit(X_train,y_train)
y_pred_rf = pipe_rf.predict(X_test)

print(classification_report(y_test, y_pred_rf))

<클래스 불균형>
              precision    recall  f1-score   support

           0       0.90      1.00      0.95        61
           1       1.00      0.53      0.70        15

    accuracy                           0.91        76
   macro avg       0.95      0.77      0.82        76
weighted avg       0.92      0.91      0.90        76



In [127]:
print('<클래스 불균형: RandomForest(균형모드)>')

from sklearn.ensemble import RandomForestClassifier

pipe_rf = Pipeline([
    ('sclaer',StandardScaler()),
    ('clf',RandomForestClassifier(class_weight='balanced',random_state=42))
])

pipe_rf.fit(X_train,y_train)
y_pred_rf = pipe_rf.predict(X_test)

print(classification_report(y_test, y_pred_rf))

<클래스 불균형: RandomForest(균형모드)>
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        61
           1       1.00      0.60      0.75        15

    accuracy                           0.92        76
   macro avg       0.96      0.80      0.85        76
weighted avg       0.93      0.92      0.91        76

